# Lab 4: Urban Heat Island Detection using Thermal Satellite Imagery

## Goal
In this exercise, you will:
1. Select and compare two thermal satellite images from Kraków (or another city that has recently undergone intensive, thoughtless concrete development) – one from **2013** and one from **2024**.
2. Each image must be acquired on a **hot summer day** (T > 27°C based on IMGW meteorological data).
3. Images must have **low cloud cover** (< 20%).
4. Visualize thermal data and analyze surface temperature differences (Urban Heat Island effect).

## Task 1: Area of Interest (AOI)
- Define an AOI over the Reduta Street area in Kraków.
- Use `ee.Geometry.Polygon` or `ee.Geometry.Rectangle`.

In [2]:
# TODO: Define AOI
import ee
ee.Authenticate()
ee.Initialize(project='ee-kamilmolo13')

aoi = ee.Geometry.Rectangle([19.791, 50.051, 19.902, 50.102]) 


Successfully saved authorization token.


## Task 2: Download and Analyze IMGW Meteorological Data
- Visit: [IMGW Archive](https://danepubliczne.imgw.pl/data/dane_pomiarowo_obserwacyjne/)
- Navigate to:
  - `dane_meteorologiczne/dobowe/synop/2013/`
  - `dane_meteorologiczne/dobowe/synop/2024/`
- Download `s_d_tmax.csv` for both years.
- Filter the rows for **station ID 12566 (e.g. Kraków-Balice)**.
- Identify days in **July or August** with **TMAX > 27°C**.

In [4]:
import pandas as pd

data_2013 = pd.read_csv('/home/mamil/Desktop/eo_lab/rs-summer-2025-labs-KamilM13-main/lab-4-data/s_d_566_2013.csv', delimiter=',', encoding="cp1250", header=None)
data_2024 = pd.read_csv('/home/mamil/Desktop/eo_lab/rs-summer-2025-labs-KamilM13-main/lab-4-data/s_d_566_2024.csv', delimiter=',', encoding="cp1250", header=None)

hot_days_2013 = data_2013[
    (data_2013[5] > 27) &
    (data_2013[3].isin([7, 8]))
]

hot_days_2024 = data_2024[
    (data_2024[5] > 27) &
    (data_2024[3].isin([7, 8]))
]

print("Hot days in 2013:")
print(hot_days_2013)

print("\nHot days in 2024:")
print(hot_days_2024)

Hot days in 2013:
            0              1     2   3   4     5   6     7   8     9   ...  \
183  350190566  KRAKÓW-BALICE  2013   7   3  28.3 NaN  12.3 NaN  21.5  ...   
184  350190566  KRAKÓW-BALICE  2013   7   4  28.3 NaN  14.2 NaN  21.9  ...   
185  350190566  KRAKÓW-BALICE  2013   7   5  27.1 NaN  13.7 NaN  19.7  ...   
190  350190566  KRAKÓW-BALICE  2013   7  10  27.7 NaN  12.9 NaN  20.9  ...   
202  350190566  KRAKÓW-BALICE  2013   7  22  27.7 NaN  11.9 NaN  20.6  ...   
206  350190566  KRAKÓW-BALICE  2013   7  26  28.9 NaN  13.9 NaN  21.6  ...   
207  350190566  KRAKÓW-BALICE  2013   7  27  30.5 NaN  14.0 NaN  23.1  ...   
208  350190566  KRAKÓW-BALICE  2013   7  28  35.0 NaN  16.9 NaN  26.3  ...   
209  350190566  KRAKÓW-BALICE  2013   7  29  35.5 NaN  18.4 NaN  27.4  ...   
210  350190566  KRAKÓW-BALICE  2013   7  30  29.6 NaN  18.1 NaN  20.0  ...   
213  350190566  KRAKÓW-BALICE  2013   8   2  29.9 NaN  13.1 NaN  22.1  ...   
214  350190566  KRAKÓW-BALICE  2013   8   3  3

## Task 3: Select Landsat 8 Images Matching These Dates
- In Earth Engine, use `LANDSAT/LC08/C02/T1_L2` collection.
- Apply filters:
  - `.filterBounds(aoi)`
  - `.filterDate()` for the matching day
  - `.filterMetadata('CLOUD_COVER', 'less_than', 20)`
- Try to find **one image from 2013** and **one image from 2024** that match your hot days list.

In [5]:
image_2013 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterBounds(aoi) \
    .filterDate('2013-07-26', '2013-08-09') \
    .sort('CLOUDY_PIXEL_PERCENTAGE') \
    .first()

image_2024 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterBounds(aoi) \
    .filterDate('2024-06-18', '2024-07-09') \
    .sort('CLOUDY_PIXEL_PERCENTAGE') \
    .first()

    

print(image_2013.getInfo())
print(image_2024.getInfo())

{'type': 'Image', 'bands': [{'id': 'SR_B1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [7961, 8051], 'crs': 'EPSG:32634', 'crs_transform': [30, 0, 303885, 0, -30, 5690715]}, {'id': 'SR_B2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [7961, 8051], 'crs': 'EPSG:32634', 'crs_transform': [30, 0, 303885, 0, -30, 5690715]}, {'id': 'SR_B3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [7961, 8051], 'crs': 'EPSG:32634', 'crs_transform': [30, 0, 303885, 0, -30, 5690715]}, {'id': 'SR_B4', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [7961, 8051], 'crs': 'EPSG:32634', 'crs_transform': [30, 0, 303885, 0, -30, 5690715]}, {'id': 'SR_B5', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 65535}, 'dimensions': [7961, 8051], 'crs': 'EPSG:32634', 'crs_transform': [30, 0, 303885, 0, -30

## Task 4: Process Thermal Band (ST_B10)
- Convert Band 10 to Brightness Temperature (Kelvin):
  `TB = ST_B10 * 0.00341802 + 149.0`
- Create a visualization of each image using the same color scale.

In [6]:
TB_im13 = image_2013.select('ST_B10').multiply(0.00341802).add(149).rename('TB')
TB_im24 = image_2024.select('ST_B10').multiply(0.00341802).add(149).rename('TB')
TB_diff = TB_im24.subtract(TB_im13).rename('TB_diff')

meta13 = TB_im13.getInfo()
meta24 = TB_im24.getInfo()
meta_diff = TB_diff.getInfo()

import geemap
viz_params = {'bands': ['TB']}
# Add Sentinel-2 images for both dates to the map using this composite.
Map = geemap.Map()
centerpoint = ee.Geometry.Point([19.974584, 50.093306])
Map.centerObject(centerpoint, zoom=14)

Map.addLayer(TB_im13, {
    'min': meta13['bands'][0]['data_type']['min'],
    'max': meta13['bands'][0]['data_type']['max'],
    'palette': ['blue', 'yellow', 'red']
}, 'Temperature 2013')

Map.addLayer(TB_im24, {
    'min': meta24['bands'][0]['data_type']['min'],
    'max': meta24['bands'][0]['data_type']['max'],
    'palette': ['blue', 'yellow', 'red']
}, 'Temperature 2024')

Map.addLayer(TB_diff, {
    'min': meta_diff['bands'][0]['data_type']['min'],
    'max': meta_diff['bands'][0]['data_type']['max'],
    'palette': ['blue', 'green', 'red']
}, 'Temperature Difference 2013-2024')

Map

Map(center=[50.093306, 19.974584], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sear…

## Task 5: Compare and Interpret
- Compare the two maps.
- Optionally calculate difference: `TB_2024 - TB_2013`
- Discuss: did the surface temperature increase in the area?
- Is there evidence of an Urban Heat Island effect related to development?


Despite development, surfer temperatures were clearly higher in 2013, especially near Krakow. There were also more Urban Heat Islands than in 2024, though some still appear in the city centre. The temperature difference map shows similar overall temperatures. It's hard to link this directly to development—some areas suggest a connection, but overall, changes over the 10 years weren’t significant. More data and analysis are needed for firm conclusions.
